# Introduction to PyTorch
### DSC 399: Advanced Applications and Interpretability of Neural Networks 

## Imports and Data Set Up

In [43]:
#############
## IMPORTS ##
#############

# For Data Set Loading and Preprocessing
# Data set for the feedforward neural networks
from sklearn.datasets import load_breast_cancer
# Train test split function from sklearn
from sklearn.model_selection import train_test_split
# Standard scaler to normalize from sklearn
from sklearn.preprocessing import StandardScaler
# Data sets for CNNs and RNNs from Tensorflow
from tensorflow.keras.datasets import mnist, imdb
# Pad sequences from Keras for NLP data formatting
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tensorflow/Keras Imports
# Import base tensorflow
import tensorflow as tf
# Import the sequential model to create neural networks
from tensorflow.keras.models import Sequential
# Import different layer types needed from Keras
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, LSTM, Embedding

# PyTorch Imports
# Import base PyTorch
import torch
# Import the neural network capabilities
import torch.nn as nn
# Import the optimizers
import torch.optim as optim

In [ ]:
#################################
## IMPORT AND FORMAT DATA SETS ##
#################################
# This data set, which predicts if a person does or does not have breast cancer,
# is from sklearn. We will import it, perform a train test split, and format the 
# data set to be used in a neural network before returning the X and y components
# of the training and test data set.
def load_breast_cancer_data(test_size=0.2):
    # Load the breast cancer dataset from sklearn
    X,y = load_breast_cancer(return_X_y=True)

    # Split the dataset into training and testing sets
    # Using the test size specified in the arguments (default of 20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    # Standardize/Normalize the features
    # Neural networks perform better with normalized features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Return the data
    return X_train, X_test, y_train, y_test

# This data set, containing images of hand written digits and the number
# being shown, will be used for convolutional neural networks
def load_mnist_data():
    # Load the MNIST dataset from tensorflow. It is already split into training
    # and test data
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    # Normalize the images to the range [0, 1] (note that for grayscale
    # images pixels range from 0 to 255).
    X_train = X_train / 255.0
    X_test = X_test / 255.0

    # Return the needed data
    return X_train, X_test, y_train, y_test

# This daat set, which contains the text from movie reviews and rather the review
# is positive or negative, will be used for the recurrent neural networks to perform
# natural language processing. Note that the data set is already tokenized. The 
# arguments are the number of unique words allowed across all review and the number of 
# words allowed in each review.
def load_imdb_data(num_words=10000, maxlen=500):
    # Load the IMDB dataset from tensorflow
    (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

    # Pad sequences to ensure uniform input size (i.e. all reviews have exactly
    # maxlen words in them). Adds zeros if the review is too short and truncates
    # if it is too long.
    X_train = pad_sequences(X_train, maxlen=maxlen)
    X_test = pad_sequences(X_test, maxlen=maxlen)

    # Return the needed data
    return X_train, X_test, y_train, y_test



## Feedforward Neural Networks

In [ ]:
####################################################
## Feedforward Neural Network in Tensorflow/Keras ##
####################################################
# Create the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(30,)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')   # binary classification
])

# Compile the model
model.compile(
    optimizer="adam",
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Show model summary
model.summary()

X_train, X_test, y_train, y_test = load_breast_cancer_data()
model.fit(X_train, y_train, epochs=20, batch_size=None, validation_split=None, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", test_acc*100, "%")

/Users/butlerju/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_13 (Dense)                │ (None, 64)             │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,209 (24.25 KB)

 Trainable params: 6,209 (24.25 KB)

 Non-trainable params: 0 (0.00 B)

Test Accuracy: 96.49122953414917 %


In [17]:
###########################################
## Feedforward Nerual Network in PyTorch ##
###########################################
class BreastCancerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(30, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()   # REQUIRED for BCELoss
        )

    def forward(self, x):
        return self.net(x)


In [18]:
class BreastCancerNet_V2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [19]:
X_train, X_test, y_train, y_test = load_breast_cancer_data()
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [20]:
model = BreastCancerNet()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 20
for epoch in range(epochs):
    model.train()

    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [10/20], Loss: 0.5855
Epoch [20/20], Loss: 0.4203


In [21]:
model.eval()
with torch.no_grad():
    probs = model(X_test)
    preds = (probs >= 0.5).float()
    accuracy = (preds == y_test).sum() / y_test.size(0)

print(f"Test Accuracy: {accuracy.item():.3f}")

Test Accuracy: 0.947


In [49]:
######################################################
## Convolutional Neural Network in Tensorflow/Keras ##
######################################################

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28,1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, kernel_size=(3, 3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation="relu"),
    Dense(10, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

X_train, X_test, y_train, y_test = load_mnist_data()

X_train = X_train[:10000]
y_train = y_train[:10000]

model.fit(X_train, y_train, epochs=5, batch_size=None, validation_split=None, verbose=0)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")


/Users/butlerju/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

Test accuracy: 0.976


Test accuracy: 0.990


In [27]:
#############################################
## Convolutional Neural Network in PyTorch ##
#############################################
class MNISTCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(64 * 5 * 5, 128),
            nn.ReLU(),
            nn.Linear(128, 10)   # logits
        )

    def forward(self, x):
        return self.model(x)

In [30]:
# Load data
X_train, X_test, y_train, y_test = load_mnist_data()

X_train = X_train[:10000]
y_train = y_train[:10000]

# Convert to PyTorch tensors and add channel dimension
X_train = torch.tensor(X_train).unsqueeze(1)  # (N, 1, 28, 28)
X_test  = torch.tensor(X_test).unsqueeze(1)

y_train = torch.tensor(y_train, dtype=torch.long)
y_test  = torch.tensor(y_test, dtype=torch.long)

model = MNISTCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

epochs = 5
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/5], Loss: 2.3016
Epoch [2/5], Loss: 2.2623
Epoch [3/5], Loss: 2.2150
Epoch [4/5], Loss: 2.1556
Epoch [5/5], Loss: 2.0832


In [31]:
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    preds = torch.argmax(outputs, dim=1)
    accuracy = (preds == y_test).float().mean()

print(f"Test accuracy: {accuracy.item():.3f}")

Test accuracy: 0.691


In [34]:
##################################################
## Recurrent Neural Network in Tensorflow/Keras ##
##################################################

num_words = 1000
maxlen = 100
X_train, X_test, y_train, y_test = load_imdb_data(num_words, maxlen)

model = Sequential([
    Embedding(input_dim=num_words, output_dim=128, input_length=maxlen),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

/Users/butlerju/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train, y_train,epochs=5,batch_size=None,validation_split=None,verbose=0)

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.7084 - loss: 0.5480
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.8271 - loss: 0.3825
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8530 - loss: 0.3386
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.8679 - loss: 0.3050
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.8775 - loss: 0.2938


In [36]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

Test accuracy: 0.841


In [37]:
#########################################
## Recurrent Neural Network in PyTorch ##
#########################################
class IMDBRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=64):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)           # (batch, seq_len, embed_dim)
        _, (h_n, _) = self.lstm(x)      # final hidden state
        x = h_n[-1]                     # (batch, hidden_dim)
        x = self.sigmoid(self.fc(x))    # probability
        return x


In [38]:
# Load data
num_words = 1000
maxlen = 100
X_train, X_test, y_train, y_test = load_imdb_data(num_words, maxlen)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.long)
X_test  = torch.tensor(X_test, dtype=torch.long)

y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test  = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [39]:
model = IMDBRNN(num_words)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5
for epoch in range(epochs):
    model.train()

    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/5], Loss: 0.6940
Epoch [2/5], Loss: 0.6920
Epoch [3/5], Loss: 0.6901
Epoch [4/5], Loss: 0.6883
Epoch [5/5], Loss: 0.6865


In [40]:
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    preds = (outputs >= 0.5).float()
    accuracy = (preds == y_test).float().mean()

print(f"Test accuracy: {accuracy.item():.3f}")

Test accuracy: 0.558
